In [ ]:
from google.colab import drive
from pathlib import Path
import scipy.io

drive.mount('/content/drive')

ROOT_PATH = Path('/content/drive/MyDrive/Colab Notebooks/ECE 2500Y')

dataset_A_path = ROOT_PATH / 'Dataset_A(days1_3).mat'
dataset_A = scipy.io.loadmat(dataset_A_path)
dataset_B_path = ROOT_PATH / 'Dataset_B(days1_3).mat'
dataset_B = scipy.io.loadmat(dataset_B_path)

In [ ]:
participant_baseline_data_A = dataset_A.get('Participants', 'Unknown')
participant_12_weeks_data_A = dataset_A.get('Participants_12weeks', 'Unknown')
participant_baseline_data_B = dataset_B.get('Participants', 'Unknown')
participant_12_weeks_data_B = dataset_B.get('Participants_12weeks', 'Unknown')

#Data generation

In [ ]:
import numpy as np
# Helpers
def extend_list(lst1, lst2):
  for i in range(len(lst1)):
    lst1[i].extend(lst2[i])
  return lst1

def get_one_day_trips(data, idx):
  return [data[0][i][idx].shape[0] for i in range(len(data[0]))]

def get_number_of_trips_in_three_days(data):
  return [get_one_day_trips(data, i) for i in range(3)]


In [ ]:
def generate_frequency_based_categorical(num_samples, data):
    unique, counts = np.unique(data, return_counts=True)
    probabilities = counts / counts.sum()
    #print(f"Unique: {unique}, Counts: {counts}, Probabilities: {probabilities}")

    return np.random.choice(unique, num_samples, p=probabilities)

def get_col(data, col_idx):
  col_lst = [data[0][i][k][m][col_idx] for i in range(len(data[0])) for k in range(len(data[0][i])) for m in range(len(data[0][i][k]))]
  return col_lst

def is_night(time_lst):
  return [1 if (0 < itm < 0.25) or (1 < itm < 1.25) else 0 for itm in time_lst]

In [ ]:
from functools import partial


# Datasets
baselines = [participant_baseline_data_A, participant_baseline_data_B]
twleve_weeks = [participant_12_weeks_data_A, participant_12_weeks_data_B]

def generate_daily_num_of_trips(data):
  day1_trips = generate_frequency_based_categorical(1, data[0])[0]
  day2_trips = generate_frequency_based_categorical(1, data[1])[0]
  day3_trips = generate_frequency_based_categorical(1, data[2])[0]
  return day1_trips, day2_trips, day3_trips

def generate_one_paitient(which_dataset):
  partial_get_cols = list(map(lambda dataset: partial(get_col, dataset), which_dataset))

  col_stacks = [[] for _ in range(6)]
  for partial_get_col in partial_get_cols:
    col_stacks = extend_list(col_stacks, list(map(partial_get_col, range(6))))

  tep = extend_list(get_number_of_trips_in_three_days(which_dataset[0]), get_number_of_trips_in_three_days(which_dataset[1]))
  day1_num_of_trips, day2_num_of_trips, day3_num_of_trips = generate_daily_num_of_trips(tep)

  day1_2_3_time_lsts = list(map(sorted, list(map(lambda x: generate_frequency_based_categorical(x, col_stacks[0]), [day1_num_of_trips, day2_num_of_trips, day3_num_of_trips]))))

  col_stacks_except_time_and_night_day1 = list(map(lambda x: generate_frequency_based_categorical(day1_num_of_trips, x), col_stacks[1:5]))
  col_stacks_except_time_and_night_day2 = list(map(lambda x: generate_frequency_based_categorical(day2_num_of_trips, x), col_stacks[1:5]))
  col_stacks_except_time_and_night_day3 = list(map(lambda x: generate_frequency_based_categorical(day3_num_of_trips, x), col_stacks[1:5]))
  col_stacks_except_time_and_night = [col_stacks_except_time_and_night_day1, col_stacks_except_time_and_night_day2, col_stacks_except_time_and_night_day3]

  is_nights = list(map(is_night, day1_2_3_time_lsts))
  col_stacks_generated_data = []
  for i in range(len(day1_2_3_time_lsts)):
    lst = [day1_2_3_time_lsts[i]] +  col_stacks_except_time_and_night[i] + [is_nights[i]]
    col_stack_one_day = np.column_stack(lst)
    col_stacks_generated_data.append(col_stack_one_day)



  #print(col_stacks_generated_data)
  return col_stacks_generated_data



In [ ]:
n = 10000 # Generate 1000 patients, 3 days data each

baseline_gen_data = [generate_one_paitient(baselines) for _ in range(n)]
twleve_weeks_gen_data = [generate_one_paitient(twleve_weeks) for _ in range(n)]

#Data prepare

##Actual Data

In [ ]:
def stack_participant_data(x):
  result_array = []
  for i in range(len(x[0])):
    combined_array = np.vstack((x[0][i][0],x[0][i][1],x[0][i][2]))
    transposed_array = combined_array.T

    new_array = np.copy(transposed_array[0])
    counter = 0
    for i in range(1, len(transposed_array[0])):
        if transposed_array[0][i] < transposed_array[0][i - 1] and counter < 3:
            new_array[i:] += 1
            counter += 1
    transposed_array[0] = new_array
    transposed_array = transposed_array[[0, 2, 3, 5]]
    result_array.append(transposed_array)

  return result_array

stacked_participant_baseline = stack_participant_data(participant_baseline_data_A) + stack_participant_data(participant_baseline_data_B)
stacked_participant_week12 = stack_participant_data(participant_12_weeks_data_A) + stack_participant_data(participant_12_weeks_data_B)

##Generated data

In [ ]:
import tensorflow as tf

def select_and_transpose_columns(data, columns_to_remove):
    processed_data = []

    for inner_list in data:
        processed_inner_list = []
        for array in inner_list:
            all_columns = np.arange(array.shape[1])
            columns_to_keep = np.setdiff1d(all_columns, columns_to_remove)
            selected_columns = array[:, columns_to_keep]
            transposed_array = selected_columns.T
            processed_inner_list.append(transposed_array)
        processed_data.append(processed_inner_list) #这里我先不rag

    return processed_data

columns = [0, 1, 4]

In [ ]:
def sum_of_specific_column(data, column_index):

    sums_of_columns = []

    for inner_list in data:
        inner_list_sum = 0
        for array in inner_list:
            inner_list_sum += np.sum(array[:, column_index])
        sums_of_columns.append(inner_list_sum/3)

    return sums_of_columns

baseline_gen_trip_label = sum_of_specific_column(baseline_gen_data, 1)
twleve_weeks_gen_trip_label = sum_of_specific_column(twleve_weeks_gen_data, 1)

baseline_gen_accLeaks_label = sum_of_specific_column(baseline_gen_data, 2)
twleve_weeks_gen_accLeaks_label = sum_of_specific_column(twleve_weeks_gen_data, 2)

baseline_gen_urges_label = sum_of_specific_column(baseline_gen_data, 3)
twleve_weeks_gen_urges_label = sum_of_specific_column(twleve_weeks_gen_data, 3)

baseline_gen_nocturia_label = sum_of_specific_column(baseline_gen_data, 5)
twleve_weeks_gen_nocturia_label = sum_of_specific_column(twleve_weeks_gen_data, 5)

In [ ]:
gen_trips_percentage_change_per_day = [(-(twelve - baseline) / baseline * 100 if baseline != 0 else 0) for twelve, baseline in zip(twleve_weeks_gen_trip_label, baseline_gen_trip_label)]
gen_accLeaks_percentage_change_per_day = [(-(twelve - baseline) / baseline * 100 if baseline != 0 else 0) for twelve, baseline in zip(twleve_weeks_gen_accLeaks_label, baseline_gen_accLeaks_label)]
gen_urges_percentage_change_per_day = [(-(twelve - baseline) / baseline * 100 if baseline != 0 else 0) for twelve, baseline in zip(twleve_weeks_gen_urges_label, baseline_gen_urges_label)]
gen_nocturia_percentage_change_per_day = [(-(twelve - baseline) / baseline * 100 if baseline != 0 else 0) for twelve, baseline in zip(twleve_weeks_gen_nocturia_label, baseline_gen_nocturia_label)]

In [ ]:
def label_numbers(arr, threshold):
    labels = []
    for i in arr:
        if i < -1 * threshold:
            labels.append(-1)
        elif -1 * threshold <= i <= threshold:
            labels.append(0)
        else:
            labels.append(1)
    return labels

# def label_numbers(arr, threshold):
#     labels = []
#     for i in arr:
#         if i < threshold:
#             labels.append(0)
#         else:
#             labels.append(1)
#     return labels

def average_position(arrays):
    averages = []
    length = len(arrays[0])
    for i in range(length):
        avg = sum(arr[i] for arr in arrays) / len(arrays)
        if avg > 0.5:
            label = 1
        else:
            label = 0
        averages.append(label)
    return averages

trips_label = label_numbers(gen_trips_percentage_change_per_day, np.average(gen_trips_percentage_change_per_day))
accLeak_label = label_numbers(gen_accLeaks_percentage_change_per_day, np.average(gen_accLeaks_percentage_change_per_day))
urges_label = label_numbers(gen_urges_percentage_change_per_day, np.average(gen_urges_percentage_change_per_day))
nocturia_label = label_numbers(gen_nocturia_percentage_change_per_day, np.average(gen_nocturia_percentage_change_per_day))
# threshold = 30
# trips_label = label_numbers(gen_trips_percentage_change_per_day, threshold)
# accLeak_label = label_numbers(gen_accLeaks_percentage_change_per_day, threshold)
# urges_label = label_numbers(gen_urges_percentage_change_per_day, threshold)
# nocturia_label = label_numbers(gen_nocturia_percentage_change_per_day, threshold)

baseline_gen_data_for_train = select_and_transpose_columns(baseline_gen_data, columns)
twleve_weeks_gen_data_for_train = select_and_transpose_columns(twleve_weeks_gen_data, columns)
t_baseline_gen_data_for_train = [np.concatenate(sublist, axis=1) for sublist in baseline_gen_data_for_train] #prepared data for train
t_twleve_weeks_gen_data_for_train = [np.concatenate(sublist, axis=1) for sublist in twleve_weeks_gen_data_for_train] #prepared data for train

gen_labels = average_position([trips_label, nocturia_label, accLeak_label, urges_label]) #prepared label for train

actual_participant_baseline = [sequence[1:] for sequence in stacked_participant_baseline] #Actual data for test
actual_participant_week12 = [sequence[1:] for sequence in stacked_participant_week12] #Actual data for test
actual_labels = [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1] #Actual labels for test

In [ ]:
indices_of_responder = [index for index, value in enumerate(gen_labels) if value == 1]
indices_of_non_responder = [index for index, value in enumerate(gen_labels) if value == 0]

responder_base_gen_trip_label = [baseline_gen_trip_label[index] for index in indices_of_responder]
non_responder_base_gen_trip_label = [baseline_gen_trip_label[index] for index in indices_of_non_responder]
responder_twleve_weeks_gen_trip_label = [twleve_weeks_gen_trip_label[index] for index in indices_of_responder]
non_responder_twleve_weeks_gen_trip_label = [twleve_weeks_gen_trip_label[index] for index in indices_of_non_responder]

responder_baseline_gen_accLeaks_label = [baseline_gen_accLeaks_label[index] for index in indices_of_responder]
non_responder_baseline_gen_accLeaks_label = [baseline_gen_accLeaks_label[index] for index in indices_of_non_responder]
responder_twleve_weeks_gen_accLeaks_label = [twleve_weeks_gen_accLeaks_label[index] for index in indices_of_responder]
non_responder_twleve_weeks_gen_accLeaks_label = [twleve_weeks_gen_accLeaks_label[index] for index in indices_of_non_responder]

responder_baseline_gen_urges_label = [baseline_gen_urges_label[index] for index in indices_of_responder]
non_responder_baseline_gen_urges_label = [baseline_gen_urges_label[index] for index in indices_of_non_responder]
responder_twleve_weeks_gen_urges_label = [twleve_weeks_gen_urges_label[index] for index in indices_of_responder]
non_responder_twleve_weeks_gen_urges_label = [twleve_weeks_gen_urges_label[index] for index in indices_of_non_responder]

responder_baseline_gen_nocturia_label = [baseline_gen_nocturia_label[index] for index in indices_of_responder]
non_responder_baseline_gen_nocturia_label = [baseline_gen_nocturia_label[index] for index in indices_of_non_responder]
responder_twleve_weeks_gen_nocturia_label = [twleve_weeks_gen_nocturia_label[index] for index in indices_of_responder]
non_responder_twleve_weeks_gen_nocturia_label = [twleve_weeks_gen_nocturia_label[index] for index in indices_of_non_responder]

In [ ]:
def get_svd(matrix):
  try:
    U, Sigma, Vh = np.linalg.svd(matrix, full_matrices=True)
  except Exception:
    Sigma = [0]
  return sum(Sigma)

def get_cond(matrix):
  try:
    return np.linalg.cond(matrix)
  except Exception:
    return np.nan

In [ ]:
E2_norm_base = [np.linalg.norm(matrix, 'fro') for matrix in t_baseline_gen_data_for_train]
sum_diag_base = [get_svd(matrix) for matrix in t_baseline_gen_data_for_train]
E2_norm_12 = [np.linalg.norm(matrix, 'fro') for matrix in t_twleve_weeks_gen_data_for_train]
sum_diag_12 = [get_svd(matrix) for matrix in t_twleve_weeks_gen_data_for_train]
ranks_base = [matrix.shape[1] for matrix in t_baseline_gen_data_for_train]
ranks_12 = [matrix.shape[1] for matrix in t_twleve_weeks_gen_data_for_train]
cond_base = [get_cond(matrix) for matrix in t_baseline_gen_data_for_train]
cond_12 = [get_cond(matrix) for matrix in t_twleve_weeks_gen_data_for_train]
#print(len(t_baseline_gen_data_for_train))

Eigenvalues, Autocorrelation Coefficients, Variance, Covariance, Singular Value, Granger Causality Statistic

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoLars
import pandas as pd

# df = pd.DataFrame({"E2_norm_base": E2_norm_base, "sum_diag_base": sum_diag_base, "E2_norm_12": E2_norm_12, "sum_diag_12": sum_diag_12, "ranks_base": ranks_base, "ranks_12": ranks_12, "cond_base": cond_base, "cond_12": cond_12, "target": gen_labels})
# df = df.replace([np.inf, -np.inf], np.nan).dropna()

df = pd.DataFrame({"E2_norm_base": E2_norm_base, "sum_diag_base": sum_diag_base, "ranks_base": ranks_base, "cond_base": cond_base, "target": gen_labels})
df = df.replace([np.inf, -np.inf], np.nan).dropna()


# X = df[['E2_norm_base', 'sum_diag_base', "E2_norm_12", "sum_diag_12", "ranks_base", "ranks_12", "cond_base", "cond_12"]]
# y = df['target']

X = df[['E2_norm_base', 'sum_diag_base', "ranks_base", "cond_base"]]
y = df['target']

In [ ]:
num_zeros = (y == 0).sum()
num_ones = (y == 1).sum()

total = len(y)

percentage_zeros = (num_zeros / total) * 100
percentage_ones = (num_ones / total) * 100

print(f"Number of non-responders (average threshold): {num_zeros} ({percentage_zeros:.2f}%)")
print(f"Number of responders (average threshold): {num_ones} ({percentage_ones:.2f}%)")

In [ ]:
model = LogisticRegression()
model.fit(X, y)

In [ ]:
# Try Lasso

from sklearn import linear_model
#reg = linear_model.Lasso(alpha=0.01)
reg = LassoLars(alpha=.01)
reg.fit(X, y)



In [ ]:
E2_norm_test = [np.linalg.norm(matrix, 'fro') for matrix in actual_participant_baseline]
sum_diag_test = [get_svd(matrix) for matrix in actual_participant_baseline]
E2_norm_12_test = [np.linalg.norm(matrix, 'fro') for matrix in actual_participant_week12]
sum_diag_12_test = [get_svd(matrix) for matrix in actual_participant_week12]
ranks_base_test = [matrix.shape[1] for matrix in actual_participant_baseline]
ranks_12_test = [matrix.shape[1] for matrix in actual_participant_week12]
cond_base_test = [get_cond(matrix) for matrix in actual_participant_baseline]
cond_12_test = [get_cond(matrix) for matrix in actual_participant_week12]

# df_test = pd.DataFrame({'E2_norm_base': E2_norm_test, 'sum_diag_base': sum_diag_test, "E2_norm_12": E2_norm_12_test, "sum_diag_12":sum_diag_12_test, "ranks_base": ranks_base_test, "ranks_12": ranks_12_test, "cond_base": cond_base_test, "cond_12": cond_12_test,"target": actual_labels})
df_test = pd.DataFrame({'E2_norm_base': E2_norm_test, 'sum_diag_base': sum_diag_test, "ranks_base": ranks_base_test, "cond_base": cond_base_test, "target": actual_labels})

In [ ]:
df_test = df_test.replace([np.inf, -np.inf], np.nan).dropna()

# X_test = df_test[['E2_norm_base', 'sum_diag_base', "E2_norm_12", "sum_diag_12", "ranks_base", "ranks_12", "cond_base", "cond_12"]]
# y_test = df_test['target']

X_test = df_test[['E2_norm_base', 'sum_diag_base', "ranks_base", "cond_base"]]
y_test = df_test['target']


In [ ]:
predictions = model.predict(X_test)

print(predictions)

In [ ]:
predictions_lasso = reg.predict(X_test)
print(predictions_lasso)

In [ ]:
def sigmoid(x,threshold=0.5):
    sigmoid_values = 1 / (1 + np.exp(-x))
    binary_values = (sigmoid_values >= threshold).astype(int)
    return binary_values

In [ ]:
predictions_binary = [sigmoid(x) for x in predictions_lasso]
#print(predictions_binary)
print(f"accuracy: {1 - sum(np.abs(predictions - y_test))/ len(y_test)}")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

model = GradientBoostingClassifier(random_state=42)
model.fit(X, y)
y_pred = model.predict(X_test)


In [ ]:
print(f"  accuracy: {accuracy_score(y_test, y_pred)}")

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', random_state=42)

model.fit(X, y)

# Predict on the test set
y_pred = model.predict(X_test)

In [ ]:
print(f"  accuracy: {accuracy_score(y_test, y_pred)}")

In [ ]:
from keras import backend as K

def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    f1 = K.mean(f1)
    return f1

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

"""
Please uncomment the below two lines if you run from the beginning.
"""
X = X.values.reshape((X.shape[0], 1, X.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_score])


In [ ]:
model.fit(X[:8000], y[:8000], epochs=50, batch_size=32, validation_data=(X[8000:], y[8000:]), verbose=1)
model.evaluate(X_test, y_test, verbose=1)

In [ ]:
loss, accuracy, f1 = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')
print(f'Test F1 Score: {f1}')

In [ ]:


model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the model with accuracy as a metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_score])


In [ ]:
model.summary()
X[:8000].shape

In [ ]:
model.fit(X[:8000], y[:8000], epochs=50, batch_size=32, validation_data=(X[8000:], y[8000:]), verbose=1)

In [ ]:
loss, accuracy, f1 = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')
print(f'Test F1 Score: {f1}')

In [ ]:
from sklearn.metrics import f1_score

y_pred = (model.predict(X_test) > 0.5).astype("int32")

f1 = f1_score(y_test, y_pred)

print("F1 Score:", f1)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

y_pred = (model.predict(X) > 0.5).astype("int32")
cm = confusion_matrix(y, y_pred)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()